In [270]:
from Bio import Entrez
from collections import OrderedDict
import xmltodict
import pandas as pd
import re
import json
import os

In [241]:
Entrez.email = "sr2661@nyu.edu"

In [306]:
title = "sdfsaf"

In [ ]:
if "PubmedArticle" in meta["PubmedArticleSet"]:
            meta = meta["PubmedArticleSet"]["PubmedArticle"]
        

In [299]:
def fetch_ids(search_term):
    query_return = Entrez.read(Entrez.egquery(term="\"{}\"".format(title)))
    response_count = int([d for d in a["eGQueryResult"] if d["DbName"] == 'pubmed'][0]["Count"])

    handle = Entrez.read(Entrez.esearch(db="pubmed"
                                        ,retmax=response_count
                                        ,term="\"{}\"".format(title)
                                       )
                        )

    id_list = handle["IdList"]
    return id_list

In [281]:
# def fetch_ids(search_term):

#     handle = Entrez.read(Entrez.esearch(db="pubmed"
# #                                         ,retmax=1000
#                                         ,term="\"{}\"".format(title)
#                                        )
#                         )

#     id_list = handle["IdList"]
#     return handle

In [303]:
def get_results(id_list):
    if isinstance(id_list,list):
        id_list = ",".join(id_list)
        fetch_result = Entrez.efetch(db="pubmed", id=id_list,retmode = "xml")

        data = fetch_result.read()
        fetch_result.close()

        xml = xmltodict.parse(data)
        meta_list = json.loads(json.dumps(xml))
        meta = meta_list["PubmedArticleSet"]["PubmedArticle"]
        return meta
    else:
        raise Exception('Input to fetch from Pubmed is not a list of IDs') 
    

In [308]:
# id_list = fetch_ids(search_term=title)
query_return = Entrez.read(Entrez.egquery(term="\"{}\"".format(title)))

In [313]:
query_return
# [d for d in query_return["eGQueryResult"] if d["DbName"] == 'pubmed'][0]
response_count = int([d for d in query_return["eGQueryResult"] if d["DbName"] == 'pubmed'][0]["Count"])
response_count

0

In [304]:
a = get_results(id_list)

In [305]:
len(a)

6843

In [200]:
def gen_pub_results(m,dataset_id):
    pm = m["MedlineCitation"]["Article"]
    pub_meta = {}
    pub_meta["title"] = pm["ArticleTitle"]
    pub_meta["datasets"] = dataset_id
    pub_meta["journal"] = pm["Journal"]["Title"]
    pub_meta["ISSN"] = pm["Journal"]["ISSN"]["#text"]
    try:
        try:
            if pm["ELocationID"]['@EIdType'] == 'doi':
                pub_meta["doi"] = pm["ELocationID"]["#text"]
        except:
            pass
        try:
            pub_meta["url"] = "https://doi.org/{}".format(pub_meta["doi"])
        except:
            pid_list = m["PubmedData"]["ArticleIdList"]["ArticleId"]
            for i in pid_list:
                for k,v in i.items():
                    if v == 'pmc':
                        pmcid = i["#text"]
                        url = "https://www.ncbi.nlm.nih.gov/pmc/articles/{}".format(pmcid)
                    else:
                        pass
        return pub_meta
    except:
        return None

In [201]:
with open('/Users/sophierand/RCCustomers/customers.json') as json_file:
    customers = json.load(json_file)
          
with open('/Users/sophierand/RCDatasets/datasets.json') as json_file:
    datasets = json.load(json_file)

In [226]:
n_ds = [d['datasets'] for d in customers if d['name'] == 'NOAA'][0]
noaa_datasets = [d for d in datasets if d['id'] in n_ds]
for d in noaa_datasets:
    d['search_terms'] = [d['title']]
    try:
        d['search_terms'].extend(d['alt_title'])
    except:
        pass

In [231]:
folder_name+"/{}.csv".format(re.sub(" ","_",s))

'/Users/sophierand/RichContextMetadata/metadata/20191228_pubmed_Sea_Level_Rise_Inundation/Sea_Level_Rise_Inundation.csv'

In [237]:
# for n in noaa_datasets:
def gen_export_md(n):
    pmd_list = []
    for s in n["search_terms"]:
        id_list = fetch_ids(search_term = s)
        meta_list = get_results(id_list)
        for m in meta_list:
            pmd = gen_pub_results(m = m , dataset_id = n["id"])
            if pmd:
                pmd_list.append(pmd)
        pmd_df = pd.DataFrame(pmd_list)
        folder_name = "/Users/sophierand/RichContextMetadata/metadata/20191228_pubmed_{}".format(re.sub(" ", "_",s))
        if not os.path.exists(folder_name):
            os.mkdir(folder_name)
        file_name = folder_name+"/{}.csv".format(re.sub(" ","_",s))
        pmd_df.to_csv(file_name, index = False)



In [238]:
for n in noaa_datasets:
    gen_export_md(n)

In [239]:
[p for p in os.listdir("/Users/sophierand/RichContextMetadata/metadata") if 'pubmed' in p]

['20191228_pubmed_ENOW_data',
 '20191228_pubmed_NEXRAD',
 '20191228_pubmed_Social_Vulnerability_Index',
 '20191228_pubmed_Global_Forecast_System',
 '20191228_pubmed_National_Flood_Hazard_Layer',
 '20191228_pubmed_US_Interagency_Elevation_Inventory',
 '20191228_pubmed_NLCD',
 '20191228_pubmed_National_Land_Cover_Database',
 '20191228_pubmed_ISD',
 '20191228_pubmed_SLR',
 '20191228_pubmed_Coastal_Flood_Frequency',
 '20191228_pubmed_ENOW',
 '20191228_pubmed_3D_Elevation_Program_Data',
 '20191228_pubmed_Sea_Level_Rise_Data',
 '20191228_pubmed_C-CAP_data',
 '20191228_pubmed_C-CAP',
 '20191228_pubmed_Integrated_Surface_Database',
 '20191228_pubmed_3DEP',
 '20191228_pubmed_Economics:_National_Ocean_Watch',
 '20191228_pubmed_SVI',
 '20191228_pubmed_SLR_Data',
 '20191228_pubmed_datums',
 '20191228_pubmed_Tidal_Datums',
 '20191228_pubmed_Next_Generation_Weather_Radar',
 '20191228_pubmed_Sea_Level_Rise_Inundation',
 '20191228_pubmed_Tide_gauge_data',
 '20191228_pubmed_Coastal_Change_Analysis_Prog